In [16]:
import pandas as pd

df = pd.read_csv("homepage-data.csv")
df = df[:10]
df.head()

,Researcher of Interest,Homepage URL
0,Li QIU,https://eeqiu.people.ust.hk/?page_id=29
1,Ling SHI,https://eesling.home.ece.ust.hk
2,Kei May LAU,https://ece.hkust.edu.hk/eekmlau
3,Chi-Keung TANG,https://ieeexplore.ieee.org/author/37087155915
4,James Tin-Yau KWOK,https://cse.hkust.edu.hk/admin/people/faculty/...


In [24]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests
from markdownify import markdownify as md
import json

load_dotenv(".env")

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

headers = {
    'accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53',
    'Accept-Language': 'en-US,en;q=0.9,it;q=0.8,es;q=0.7',
    'referer': 'https://www.google.com/',
    'cookie': '94880A328F459C4F'
}

def process_homepage_text(text: str):
    response = client.chat.completions.create(
        messages=[{
            "role": "system",
            "content": """Extract the following qualities, with justification in a JSON formatted output. 
Try to infer these from the way the author writes about themselves. JSON output format: 
{
    "friendliness-justification": "Explain your friendliness quote with citations",
    "friendliness": "Scale of 1-10, how much they exhibit traits of cameraderie, teamwork and collaboration. 10 being heavy-emphasis on friendliness.",
    "we-vs-i-justification": "Explain your we-vs-i quote with citations",
    "we-vs-i": "Scale of 1-10, how much they talk about themselves vs their collaborators. 10 being heavy-emphasis on we.",
    "risk-averse-justification": "Explain your risk-averse quote with citations",
    "risk-averse": "Scale of 1-10, how they feel about taking risks. 10 being very risk-averse.",
    "career-orientation-justification": "Explain your career-orientation quote with citations",
    "career-orientation": "How much a person cares about their career. 10 being very career-oriented.",
}"""
        }, {
            "role": "user",
            "content": text
        }],
        response_format={
            "type": "json_object"
        },
        model="gpt-4o"
    )
    response = json.loads(response.choices[0].message.content)
    return response


In [25]:
from rich import print

# TODO: append university profile w/ personal website text
# TODO: make sure that results aren't correlated w/ more text (more personable because they wrote more) 

for researcher_file in os.listdir("data/websites"):
    researcher_text = open(f"data/websites/{researcher_file}").read()
    print("Processing", researcher_file)
    print("Researcher text: ", researcher_text)
    print(process_homepage_text(researcher_text))
    

Processing james-tin-yau-kwok.txt

Researcher text:  
James Kwok is a Professor in the Department of Computer Science and Engineering, Hong Kong University of Science 
and Technology. He is an IEEE Fellow.

Prof Kwok received his B.Sc. degree in Electrical and Electronic Engineering from the University of Hong Kong and 
his Ph.D. degree in computer science from the Hong Kong University of Science and Technology. He then joined the 
Department of Computer Science, Hong Kong Baptist University as an Assistant Professor. He returned to the Hong 
Kong University of Science and Technology and is now a Professor in the Department of Computer Science and 
Engineering. He is serving / served as an Associate Editor for the IEEE Transactions on Neural Networks and 
Learning Systems, Neural Networks, Neurocomputing, Artificial Intelligence Journal, International Journal of Data 
Science and Analytics, and on the Editorial Board of Machine Learning. He is also serving as Senior Area Chairs of 
major machine learning / AI conferences including NeurIPS, ICML, ICLR, IJCAI, and as Area Chairs of conferences 
including AAAI and ECML. He is on the IJCAI Board of Trustees. He is recognized as the Most Influential Scholar 
Award Honorable Mention for "outstanding and vibrant contributions to the field of AAAI/IJCAI between 2009 and 
2019". Prof Kwok is the IJCAI-2025 Program Chair.

{
    'friendliness-justification': "The text does not provide direct evidence of friendliness; however, Prof Kwok's 
engagement in various collaborative editorial roles and conferences suggests a degree of collegiality and 
teamwork.",
    'friendliness': '6',
    'we-vs-i-justification': "Prof Kwok's biography predominantly focuses on his personal achievements and roles 
without mentioning collaboration with others, indicating a more individual-focused perspective.",
    'we-vs-i': '4',
    'risk-averse-justification': "Prof Kwok's choice of an academic and editorial career path, which emphasizes 
stability and recognition over uncertainty, suggests a moderate level of risk aversion.",
    'risk-averse': '7',
    'career-orientation-justification': 'The detailed career achievements and roles in prestigious journals and 
conferences demonstrate a high level of career orientation and dedication to professional advancement.',
    'career-orientation': '9'
}

Processing li-qiu.txt

Researcher text:  Prof. Li Qiu is currently a professor in the Department of Electronic and Computer Engineering, 
at the Hong Kong University of Science and Technology.

He was born in the city of Changsha (the capital of Hunan province) in the People’s Republic of China. He received 
his Bachelor’s degree from Hunan University in 1981 and his Masters and PhD degrees from the University of Toronto 
in 1987 and 1990 respectively.

Since 1990, Dr. Li Qiu has held research and teaching positions at the University of Toronto, the Canadian Space 
Agency, the University of Waterloo, the Fields Institute for Research in Mathematical Sciences, and the University 
of Minnesota. He joined the Hong Kong University of Science and Technology in 1993 and has also held visiting 
positions at Zhejiang University, Australian Defense Force Academy, and Harbin Institute of Technology.

Dr. Li Qiu is a fellow of IEEE and a fellow of IFAC. He was a member of the IEEE Control Systems Society Board of 
Governors in 2012. He served as a distinguished lecturer of the IEEE Control Systems Society from 2007 to 2009 and 
the general chair of the 7th Asian Control Conference which was held in Hong Kong in August 2009. He has served as 
an associate editor of the publications IEEE Transactions on Automatic Control and Automatica. He is now a member 
of the editorial board of International Journal of Control.

...

Researcher text:  Chi Keung Tang received the MSc and PhD degrees in computer science from the University of 
Southern California, Los Angeles, in 1999 and 2000, respectively.

Since 2000, he has been with the Department of Computer Science and Engineering at the Hong Kong University of 
Science and Technology where he is currently a full professor.

He was on sabbatical at University of California, Los Angeles, in 2008. He was an adjunct researcher at the Visual 
Computing Group of Microsoft Research Asia. His research areas are computer vision, computer graphics and machine 
learning. He was an associate editor of IEEE Transactions on Pattern Analysis and Machine Intelligence (TPAMI) and 
was on the editorial board of International Journal of Computer Vision (IJCV). He served as an area chair for ACCV 
2006, ICCV 2007, ICCV 2009, ICCV 2011, ICCV 2015 and ICCV 2017, and as a technical papers committee member for the 
inaugural SIGGRAPH Asia 2008, SIGGRAPH 2011, SIGGRAPH Asia 2011, SIGGRAPH 2012, SIGGRAPH Asia 2012, SIGGRAPH Asia 
2014 and SIGGRAPH Asia 2015. He is a senior member of the IEEE Computer Society.

{
    'friendliness-justification': "Chi Keung Tang's background does not provide direct information on personal 
friendliness or mention collaborations aside from professional roles.",
    'friendliness': '3',
    'we-vs-i-justification': "The text focuses primarily on Chi Keung Tang's individual achievements and roles, 
with limited reference to collaboration or teamwork, as evidenced by the description of his academic and 
professional history without mention of joint efforts.",
    'we-vs-i': '3',
    'risk-averse-justification': 'The document does not provide enough information to assess his attitude towards 
risk-taking, focusing instead on his established academic and professional roles.',
    'risk-averse': '5',
    'career-orientation-justification': 'Chi Keung Tang appears highly career-oriented, indicated by his transition
from a PhD to a professorship, multiple editorial and organizational roles, and involvement with prestigious 
conferences and academic institutions.',
    'career-orientation': '9'
}

Processing kei-may-lau.txt

Researcher text:  PhD, Rice University
Fellow, IEEE
Fellow, OSA
Fellow, Hong Kong Academy of Engineering Sciences
Fellow, CSR

 

Professor Kei May Lau was born in Hong Kong and received all her pre-college education (K-12) from the Pui Ching 
Middle School.  She received the B.S. and M.S. degrees in physics from University of Minnesota, Minneapolis, in 
1976 and 1977 respectively, and the Ph.D. Degree in Electrical Engineering from Rice University, Houston, Texas, in
1981.

From 1980 to 1982, she was a Senior Engineer at M/A-COM Gallium Arsenide Products, Inc., where she worked on 
epitaxial growth of GaAs for microwave components and development of mm-wave devices. In the fall of 1982, she 
joined the faculty of the Electrical and Computer Engineering Department at the University of 
Massachusetts/Amherst, where she became a full professor in 1993. She initiated metalorganic chemical vapor 
deposition (MOCVD), compound semiconductor materials and devices programs at UMass. Her research group performed 
studies on heterostructures, quantum wells, strained-layers, III-V selective epitaxy, as well as high-frequency and
photonic devices. Professor Lau spent her first sabbatical leave in 1989 at the MIT Lincoln Laboratory and worked 
with the Electro-optical Devices Group. She developed acoustic sensors at the DuPont Central Research and 
Development Laboratory in Wilmington, Delaware during her second sabbatical leave ('95-'96). In the fall of 1998, 
she was a visiting professor at the Hong Kong University of Science and Technology (HKUST), where she joined the 
regular faculty since the summer of 2000. She established the Photonics Technology Center for R&D effort in III-V 
and wide band-gap semiconductor materials and devices.

Professor Lau is a Fellow of the IEEE, OSA, and the Hong Kong Academy of Engineering Sciences. She served on the 
IEEE Electron Devices Society Administrative Committee and was an Editor of the IEEE Transactions on Electron 
Devices (1996-2002), an Associate Editor for the Journal of Crystal Growth and Applied Physics Letters. She is now 
an Editor of the IEEE Electron Device Letters.

{
    'friendliness-justification': 'The text does not provide direct evidence regarding friendliness, but being a 
Fellow of multiple prestigious organizations suggests collaborative and collegial qualities.',
    'friendliness': '7',
    'we-vs-i-justification': "Professor Lau uses terms like 'her research group' and 'she established,' indicating 
a balance with a slight emphasis on personal contributions over collaborative ones.",
    'we-vs-i': '6',
    'risk-averse-justification': 'The text shows a pattern of innovation, such as initiating new programs and 
establishing centers, which suggests a moderate level of risk-taking.',
    'risk-averse': '4',
    'career-orientation-justification': 'The extensive detail on her career achievements, positions, and 
contributions, along with her status as a Fellow in several prestigious organizations, highlights a strong focus on
her career.',
    'career-orientation': '10'
}

In [ ]:
# TODO: save to HK-month-analysis-data